# Set Up 

In [1]:
import sys
import csv 
csv.field_size_limit(sys.maxsize)

if '..' not in sys.path:
    sys.path.append('..')

import os
import logging
import pandas as pd 
from dotenv import load_dotenv
from RAG.rag_agent import RAGAgent 
from RAG.rag_searcher import RAGSearcher
from utils.clients import create_chat_client, create_embed_client 

load_dotenv(override=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

True

In [2]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,  
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('../logs/rag_agent.log')  # Remove StreamHandler to prevent console output
    ]
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)  

# Load Data

In [3]:
df_benchmark = pd.read_csv("../data/longbench_filtered.csv", delimiter="§", engine="python")
df_context_asc = pd.read_csv("../data/longbench_context_chunked_asc.csv", delimiter="§", engine="python")
df_context_simple = pd.read_csv("../data/longbench_context_chunked_simple.csv", delimiter="§", engine="python")

# Initialise RAG Agent

In [4]:
chat_client = create_chat_client()
embed_client = create_embed_client()

rag_agent = RAGAgent(
    chat_client=chat_client,
    embed_client=embed_client,
    chat_model=os.getenv("MODEL_NAME"),
    searcher=RAGSearcher(),
    max_tokens=130000,
    temperature=0.0
)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

# Demo

In [5]:
df_benchmark.iloc[0]

_id                                               66f36490821e116aacb2cc22
domain                                                  Single-Document QA
sub_domain                                                       Financial
difficulty                                                            easy
length                                                               short
question                 According to the report, how to promote the co...
choice_A                 Through technology empowerment, change the way...
choice_B                 Establish new types of courts, such as intelle...
choice_C                 Improve the work ability of office staff and s...
choice_D                 Use advanced information systems to improve th...
answer                                                                   D
context                  Contents\nPreface.\n.............................
context_tokens                                                       38133
within_context_window    

In [10]:
import random
n = random.randint(0, len(df_benchmark))

id = df_benchmark.iloc[n]["_id"]
question = df_benchmark.iloc[n]["question"]
choice_a = df_benchmark.iloc[n]["choice_A"]
choice_b = df_benchmark.iloc[n]["choice_B"]
choice_c = df_benchmark.iloc[n]["choice_C"]
choice_d = df_benchmark.iloc[n]["choice_D"]
correct_answer = df_benchmark.iloc[n]["answer"]

In [11]:
df_context_simple[df_context_simple["_id"] == id]

,_id,chunk_id,chunk_text,embeddings
28413,66f4f87b821e116aacb31d62,0,et5 user manual user manual 2024 / 9 / 26 13 :...,"[[-0.032562255859375, -0.0107574462890625, -0...."
28414,66f4f87b821e116aacb31d62,1,: this is to advise you on how to avoid possib...,"[[-0.03607177734375, 0.003021240234375, -0.054..."
28415,66f4f87b821e116aacb31d62,2,headrest adjustment seat massage mode seat hea...,"[[-0.045745849609375, -0.03900146484375, -0.06..."
28416,66f4f87b821e116aacb31d62,3,the hood instrument panel fuse box rear trunk ...,"[[-0.03729248046875, -0.01523590087890625, -0...."
28417,66f4f87b821e116aacb31d62,4,the windows at the same time ; release the unl...,"[[-0.0181121826171875, -0.0163116455078125, -0..."
...,...,...,...,...
28671,66f4f87b821e116aacb31d62,258,the vehicle should be undertaken by profession...,"[[-0.036224365234375, -0.0089263916015625, -0...."
28672,66f4f87b821e116aacb31d62,259,"if you need the vehicle to be towed, please ca...","[[0.01654052734375, -0.01369476318359375, 0.00..."
28673,66f4f87b821e116aacb31d62,260,. when the parking brake is released and the v...,"[[-0.01065826416015625, -0.022430419921875, -0..."
28674,66f4f87b821e116aacb31d62,261,"hours ), monitor it for one more hour to ensur...","[[-0.0087738037109375, -0.00901031494140625, -..."


In [12]:
def format_question(question: str, choice_a: str, choice_b: str, choice_c: str, choice_d: str) -> str:
    return f"Question: {question}\nA: {choice_a}\nB: {choice_b}\nC: {choice_c}\nD: {choice_d}"

formatted_question = format_question(question, choice_a, choice_b, choice_c, choice_d)
print(formatted_question)

Question: In a NIO ET5, if you want the vehicle to automatically lock every time you leave without pressing the key, what should you do?
A: In the settings menu on the central control screen, go to "Vehicle Settings," find "Lock Settings," and enable the "Auto Lock" feature.
B: In the settings menu on the central control screen, go to "Safety Settings," find the "Auto Lock When Leaving Vehicle" option, and enable it.
C: Hold down the lock button on the key for 10 seconds until you hear a beep, and the vehicle will automatically enable the auto-lock feature.
D: In the settings menu on the central control screen, go to "Driver Assistance Settings," find the "Auto Lock" option, and enable it.


In [13]:
llm_answer = rag_agent.generate_response(formatted_question, df_context_simple, id, top=3)
print(llm_answer)
print(f"LLM answer == Correct answer: {llm_answer == correct_answer}")

A
LLM answer == Correct answer: True


# Benchmarking Workflow